## Create a landmask for SeaWIFS climatology

Author: Frank Bryan (bryan@ucar.edu ) 

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
## Get the land/water mask
path_in = '../ncfiles/'
file_in = 'LandWater15ARC.nc'
ds_hires = xr.open_dataset(path_in+file_in)
ds_hires

In [ ]:
## Get some SeaWIFS data (only need the coordinates)
path_chl = '../ncfiles/'
file_chl = 'S19972442010273.L3m_MC_CHL_chlor_a_9km.nc'
ds_chl = xr.open_dataset(path_chl+file_chl)
ds_chl

In [ ]:
## Subsample the 15 arcsecond (1/240 deg) mask grid to the 1/12 deg SeaWIFS grid (approx)
Mask_lores = ds_hires['LandWater'].sel(
                lon=ds_chl['lon'],
                lat=ds_chl['lat'],method='nearest').astype('int')

In [ ]:
## Create a corresponding mask of all ocean areas (shallow=0,shelf=6,deep=7)
Ocean_lores = xr.where(Mask_lores==0,1,0)
Ocean_lores = xr.where(Mask_lores>=6,1,Ocean_lores)
Ocean_lores = Ocean_lores.astype('int')

## Create the reciprocal land mask
Land_lores = (1 - Ocean_lores).astype('int')

In [ ]:
# Put the masks in a dataset
ds_out = xr.Dataset({"LandWater" : Mask_lores})

ds_out['Ocean'] = Ocean_lores
ds_out['Ocean'].attrs['long_name'] = 'Ocean Mask'
ds_out['Ocean'].attrs['flag_values'] = '0, 1'
ds_out['Ocean'].attrs['flag_meanings'] = '0=Land or Inland Water, 1=Ocean'

ds_out['Land'] = Land_lores
ds_out['Land'].attrs['long_name'] = 'Land Mask'
ds_out['Land'].attrs['flag_values'] = '0, 1'
ds_out['Land'].attrs['flag_meanings'] = '1=Land or Inland Water, 0=Ocean'

In [ ]:
# Tweak the coordinates to agree exactly with SeaWIFS 
ds_out.coords['lon'] = ds_chl['lon']
ds_out.coords['lat'] = ds_chl['lat']

In [ ]:
# Add a history attribute
from datetime import date
today = date.today()
dstr = today.strftime("%Y-%m-%d")
astr = 'Created ' + dstr + ' from input file ' + path_in+file_in
astr
ds_out.attrs['history'] = astr

In [ ]:
ds_out

In [ ]:
ds_out['Ocean'].plot(robust=True)

In [ ]:
fileout = '../ncfiles/LandWater_SeaWIFS.nc'
ds_out.to_netcdf(path=fileout)